# Import the necessary libraries

In [8]:
# !pip install torchvision matplotlib

In [9]:
# !pip install tqdm pathlib

In [6]:
import torch
import torchvision.datasets as datasets 
import torchvision.transforms as transforms
import torch.nn as nn
import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path
import os

# Load the MNIST dataset

In [10]:
# Make torch deterministic
_ = torch.manual_seed(0)

In [11]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

# Load the MNIST dataset
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
# Create a dataloader for the training
train_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=10, shuffle=True)

# Load the MNIST test set
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(mnist_testset, batch_size=10, shuffle=True)

# Define the device
device = "cpu"

100%|██████████| 9.91M/9.91M [00:03<00:00, 2.84MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 116kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 940kB/s] 
100%|██████████| 4.54k/4.54k [00:00<?, ?B/s]


# Define the model

In [12]:
class VerySimpleNet(nn.Module):
    def __init__(self, hidden_size_1=100, hidden_size_2=100):
        super(VerySimpleNet,self).__init__()
        self.linear1 = nn.Linear(28*28, hidden_size_1) 
        self.linear2 = nn.Linear(hidden_size_1, hidden_size_2) 
        self.linear3 = nn.Linear(hidden_size_2, 10)
        self.relu = nn.ReLU()

    def forward(self, img):
        x = img.view(-1, 28*28)
        x = self.relu(self.linear1(x))
        x = self.relu(self.linear2(x))
        x = self.linear3(x)
        return x

In [13]:
net = VerySimpleNet().to(device)

# Train the model

In [14]:
def train(train_loader, net, epochs=5, total_iterations_limit=None):
    cross_el = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

    total_iterations = 0

    for epoch in range(epochs):
        net.train()

        loss_sum = 0
        num_iterations = 0

        data_iterator = tqdm(train_loader, desc=f'Epoch {epoch+1}')
        if total_iterations_limit is not None:
            data_iterator.total = total_iterations_limit
        for data in data_iterator:
            num_iterations += 1
            total_iterations += 1
            x, y = data
            x = x.to(device)
            y = y.to(device)
            optimizer.zero_grad()
            output = net(x.view(-1, 28*28))
            loss = cross_el(output, y)
            loss_sum += loss.item()
            avg_loss = loss_sum / num_iterations
            data_iterator.set_postfix(loss=avg_loss)
            loss.backward()
            optimizer.step()

            if total_iterations_limit is not None and total_iterations >= total_iterations_limit:
                return
            
def print_size_of_model(model):
    torch.save(model.state_dict(), "temp_delme.p")
    print('Size (KB):', os.path.getsize("temp_delme.p")/1e3)
    os.remove('temp_delme.p')

MODEL_FILENAME = 'simplenet_ptq.pt'

if Path(MODEL_FILENAME).exists():
    net.load_state_dict(torch.load(MODEL_FILENAME))
    print('Loaded model from disk')
else:
    train(train_loader, net, epochs=1)
    # Save the model to disk
    torch.save(net.state_dict(), MODEL_FILENAME)

Epoch 1: 100%|██████████| 6000/6000 [00:39<00:00, 151.13it/s, loss=0.223]


# Define the testing loop

In [16]:
def test(model: nn.Module, total_iterations: int = None):
    correct = 0
    total = 0

    iterations = 0

    model.eval()

    with torch.no_grad():
        for data in tqdm(test_loader, desc='Testing'):
            x, y = data
            x = x.to(device)
            y = y.to(device)
            output = model(x.view(-1, 784))
            for idx, i in enumerate(output):
                if torch.argmax(i) == y[idx]:
                    correct +=1
                total +=1
            iterations += 1
            if total_iterations is not None and iterations >= total_iterations:
                break
    print(f'Accuracy: {round(correct/total, 3)}')

# Print weights and size of the model before quantization

In [17]:
# Print the weights matrix of the model before quantization
print('Weights before quantization')
print(net.linear1.weight)
print(net.linear1.weight.dtype)

Weights before quantization
Parameter containing:
tensor([[ 0.0088,  0.0282, -0.0204,  ...,  0.0310,  0.0128,  0.0111],
        [-0.0206, -0.0158, -0.0112,  ..., -0.0211, -0.0068, -0.0307],
        [ 0.0235,  0.0585,  0.0103,  ...,  0.0233,  0.0448,  0.0516],
        ...,
        [ 0.0497,  0.0534,  0.0188,  ...,  0.0134,  0.0327, -0.0043],
        [-0.0118, -0.0037,  0.0284,  ...,  0.0087,  0.0087,  0.0082],
        [ 0.0326,  0.0273,  0.0131,  ...,  0.0495,  0.0003,  0.0204]],
       requires_grad=True)
torch.float32


In [18]:
print('Size of the model before quantization')
print_size_of_model(net)

Size of the model before quantization
Size (KB): 360.998


In [19]:
print(f'Accuracy of the model before quantization: ')
test(net)

Accuracy of the model before quantization: 


Testing: 100%|██████████| 1000/1000 [00:01<00:00, 504.82it/s]

Accuracy: 0.961


# Insert min-max observers in the model

In [20]:
class QuantizedVerySimpleNet(nn.Module):
    def __init__(self, hidden_size_1=100, hidden_size_2=100):
        super(QuantizedVerySimpleNet,self).__init__()
        self.quant = torch.quantization.QuantStub()
        self.linear1 = nn.Linear(28*28, hidden_size_1) 
        self.linear2 = nn.Linear(hidden_size_1, hidden_size_2) 
        self.linear3 = nn.Linear(hidden_size_2, 10)
        self.relu = nn.ReLU()
        self.dequant = torch.quantization.DeQuantStub()

    def forward(self, img):
        x = img.view(-1, 28*28)
        x = self.quant(x)
        x = self.relu(self.linear1(x))
        x = self.relu(self.linear2(x))
        x = self.linear3(x)
        x = self.dequant(x)
        return x

In [21]:
net_quantized = QuantizedVerySimpleNet().to(device)
# Copy weights from unquantized model
net_quantized.load_state_dict(net.state_dict())
net_quantized.eval()

net_quantized.qconfig = torch.ao.quantization.default_qconfig
net_quantized = torch.ao.quantization.prepare(net_quantized) # Insert observers
net_quantized

QuantizedVerySimpleNet(
  (quant): QuantStub(
    (activation_post_process): MinMaxObserver(min_val=inf, max_val=-inf)
  )
  (linear1): Linear(
    in_features=784, out_features=100, bias=True
    (activation_post_process): MinMaxObserver(min_val=inf, max_val=-inf)
  )
  (linear2): Linear(
    in_features=100, out_features=100, bias=True
    (activation_post_process): MinMaxObserver(min_val=inf, max_val=-inf)
  )
  (linear3): Linear(
    in_features=100, out_features=10, bias=True
    (activation_post_process): MinMaxObserver(min_val=inf, max_val=-inf)
  )
  (relu): ReLU()
  (dequant): DeQuantStub()
)

# Calibrate the model using the test set

In [22]:
test(net_quantized)

Testing: 100%|██████████| 1000/1000 [00:02<00:00, 435.29it/s]

Accuracy: 0.961


In [23]:
print(f'Check statistics of the various layers')
net_quantized

Check statistics of the various layers


QuantizedVerySimpleNet(
  (quant): QuantStub(
    (activation_post_process): MinMaxObserver(min_val=-0.4242129623889923, max_val=2.821486711502075)
  )
  (linear1): Linear(
    in_features=784, out_features=100, bias=True
    (activation_post_process): MinMaxObserver(min_val=-53.75807189941406, max_val=37.913963317871094)
  )
  (linear2): Linear(
    in_features=100, out_features=100, bias=True
    (activation_post_process): MinMaxObserver(min_val=-25.207971572875977, max_val=24.764205932617188)
  )
  (linear3): Linear(
    in_features=100, out_features=10, bias=True
    (activation_post_process): MinMaxObserver(min_val=-32.0723762512207, max_val=19.65166664123535)
  )
  (relu): ReLU()
  (dequant): DeQuantStub()
)

# Quantize the model using the statistics collected

In [24]:
net_quantized = torch.ao.quantization.convert(net_quantized)

In [25]:
print(f'Check statistics of the various layers')
net_quantized

Check statistics of the various layers


QuantizedVerySimpleNet(
  (quant): Quantize(scale=tensor([0.0256]), zero_point=tensor([17]), dtype=torch.quint8)
  (linear1): QuantizedLinear(in_features=784, out_features=100, scale=0.7218270301818848, zero_point=74, qscheme=torch.per_tensor_affine)
  (linear2): QuantizedLinear(in_features=100, out_features=100, scale=0.39348170161247253, zero_point=64, qscheme=torch.per_tensor_affine)
  (linear3): QuantizedLinear(in_features=100, out_features=10, scale=0.4072759449481964, zero_point=79, qscheme=torch.per_tensor_affine)
  (relu): ReLU()
  (dequant): DeQuantize()
)

# Print weights of the model after quantization

In [26]:
# Print the weights matrix of the model after quantization
print('Weights after quantization')
print(torch.int_repr(net_quantized.linear1.weight()))

Weights after quantization
tensor([[ 2,  6, -5,  ...,  7,  3,  2],
        [-5, -4, -2,  ..., -5, -2, -7],
        [ 5, 13,  2,  ...,  5, 10, 11],
        ...,
        [11, 12,  4,  ...,  3,  7, -1],
        [-3, -1,  6,  ...,  2,  2,  2],
        [ 7,  6,  3,  ..., 11,  0,  5]], dtype=torch.int8)


# Compare the dequantized weights and the original weights

In [27]:
print('Original weights: ')
print(net.linear1.weight)
print('')
print(f'Dequantized weights: ')
print(torch.dequantize(net_quantized.linear1.weight()))
print('')

Original weights: 
Parameter containing:
tensor([[ 0.0088,  0.0282, -0.0204,  ...,  0.0310,  0.0128,  0.0111],
        [-0.0206, -0.0158, -0.0112,  ..., -0.0211, -0.0068, -0.0307],
        [ 0.0235,  0.0585,  0.0103,  ...,  0.0233,  0.0448,  0.0516],
        ...,
        [ 0.0497,  0.0534,  0.0188,  ...,  0.0134,  0.0327, -0.0043],
        [-0.0118, -0.0037,  0.0284,  ...,  0.0087,  0.0087,  0.0082],
        [ 0.0326,  0.0273,  0.0131,  ...,  0.0495,  0.0003,  0.0204]],
       requires_grad=True)

Dequantized weights: 
tensor([[ 0.0090,  0.0270, -0.0225,  ...,  0.0315,  0.0135,  0.0090],
        [-0.0225, -0.0180, -0.0090,  ..., -0.0225, -0.0090, -0.0315],
        [ 0.0225,  0.0585,  0.0090,  ...,  0.0225,  0.0450,  0.0495],
        ...,
        [ 0.0495,  0.0540,  0.0180,  ...,  0.0135,  0.0315, -0.0045],
        [-0.0135, -0.0045,  0.0270,  ...,  0.0090,  0.0090,  0.0090],
        [ 0.0315,  0.0270,  0.0135,  ...,  0.0495,  0.0000,  0.0225]])



# Print size and accuracy of the quantized model

In [28]:
print('Size of the model after quantization')
print_size_of_model(net_quantized)

Size of the model after quantization
Size (KB): 95.394


In [29]:
print('Testing the model after quantization')
test(net_quantized)

Testing the model after quantization


Testing: 100%|██████████| 1000/1000 [00:02<00:00, 432.25it/s]

Accuracy: 0.961
